In [2]:
import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

In [3]:
import os
import sys
sys.path.append('../')

import pandas as pd
pd.set_option('display.max_columns', 200)

import src.generate_report as report_gen

# Testing the Report Generation

In [4]:
report = report_gen.Report("04272022")
report.run()

KeyboardInterrupt: 

In [ ]:
report.get_winningest_team(min_games=5)

: 

In [5]:
d = report.get_simplified_results_per_player()
for p, stats in d.items():
    print(p)
    print(stats)

Brandon
{'Serves': '8.8', 'Aces': '0.9', 'Missed Serves': '0.7', 'Received': '8.4', 'Swings': '8.5', 'Swing Kills': '3.4', 'Blocks': '0.6', 'Block Kills': '0.1', 'Bump Kills': '0.5', 'Hitting Errors': '1.0', 'Errors': '2.3', 'Serving Percentage': '93.5%', 'Kills': '4.0', 'Hitting Efficiency': '39.6%', 'Serve Receive Rating': '1.9', 'Effectiveness': '2.5', 'N': '34', 'Win Rate': '41.2%', 'Ace:Error': '1.3'}
Bruno
{'Serves': '9.5', 'Aces': '0.7', 'Missed Serves': '0.8', 'Received': '7.4', 'Swings': '9.5', 'Swing Kills': '3.2', 'Blocks': '0.6', 'Block Kills': '0.1', 'Bump Kills': '0.1', 'Hitting Errors': '1.1', 'Errors': '2.3', 'Serving Percentage': '92.6%', 'Kills': '3.4', 'Hitting Efficiency': '33.9%', 'Serve Receive Rating': '2.2', 'Effectiveness': '1.8', 'N': '31', 'Win Rate': '61.3%', 'Ace:Error': '0.9'}
Carlos
{'Serves': '8.3', 'Aces': '0.8', 'Missed Serves': '1.1', 'Received': '8.1', 'Swings': '10.3', 'Swing Kills': '3.9', 'Blocks': '1.0', 'Block Kills': '0.6', 'Bump Kills': '0.3',

In [ ]:
d["Brandon"]["N"]

# Getting Fitbit Data
A test to see if we can snag Fitbit data. The documentation for this process is provided [here](https://towardsdatascience.com/using-the-fitbit-web-api-with-python-f29f119621ea).

In [36]:
import gather_keys_oauth2 as Oauth2
import pandas as pd
import fitbit
from datetime import datetime
from src.settings import FITBIT_CLIENT_ID, FITBIT_CLIENT_SECRET

The `gather_keys_oauth2` can be found on this Github [repo](https://github.com/mGalarnyk/Python_Tutorials/blob/master/Apis/Fitbit/Fitbit_API.ipynb).

## Accessing the Database

In [26]:
server=Oauth2.OAuth2Server(FITBIT_CLIENT_ID, FITBIT_CLIENT_SECRET)
server.browser_authorize()
ACCESS_TOKEN=str(server.fitbit.client.session.token['access_token'])
REFRESH_TOKEN=str(server.fitbit.client.session.token['refresh_token'])
auth2_client=fitbit.Fitbit(FITBIT_CLIENT_ID,FITBIT_CLIENT_SECRET,oauth2=True,access_token=ACCESS_TOKEN,refresh_token=REFRESH_TOKEN)

[06/Aug/2022:17:15:56] ENGINE Listening for SIGTERM.
[06/Aug/2022:17:15:56] ENGINE Listening for SIGHUP.
[06/Aug/2022:17:15:56] ENGINE Listening for SIGUSR1.
[06/Aug/2022:17:15:56] ENGINE Bus STARTING
[06/Aug/2022:17:15:56] ENGINE Started monitor thread 'Autoreloader'.
[06/Aug/2022:17:15:57] ENGINE Serving on http://127.0.0.1:8080
[06/Aug/2022:17:15:57] ENGINE Bus STARTED


127.0.0.1 - - [06/Aug/2022:17:15:59] "GET /?code=b0d1052485652054f52c23215955b84d7faee552&state=TuaLTIjxMQOkH3XvXepksLkmzHgl83 HTTP/1.1" 200 122 "" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Safari/605.1.15"


[06/Aug/2022:17:16:00] ENGINE Bus STOPPING
[06/Aug/2022:17:16:00] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('127.0.0.1', 8080)) shut down
[06/Aug/2022:17:16:00] ENGINE Stopped thread 'Autoreloader'.
[06/Aug/2022:17:16:00] ENGINE Bus STOPPED
[06/Aug/2022:17:16:00] ENGINE Bus EXITING
[06/Aug/2022:17:16:00] ENGINE Bus EXITED
[06/Aug/2022:17:16:00] ENGINE Waiting for child threads to terminate...


In [66]:
def get_fb_data(y, m, d, resource):
    """
    Gets the fb data as a DataFrame from the given day
    """
    date = datetime(y, m, d)
    try:
        raw = auth2_client.intraday_time_series(f'activities/{resource}', date, detail_level='1sec')
    except Exception as e:
        print(e)
        raw = auth2_client.intraday_time_series(f'activities/{resource}', date, detail_level='1min')

    data =  pd.DataFrame(raw[f"activities-{resource}-intraday"]["dataset"])
    ts = []
    for t in data["time"]:
        t = pd.to_datetime(t)
        ts.append(datetime(y,m,d,t.hour,t.minute,t.second))
        
    data["time"] = ts
    return data.set_index("time")

In [67]:
get_fb_data(2022,8,5,"heart")

,value
time,
2022-08-05 00:00:03,67
2022-08-05 00:00:08,68
2022-08-05 00:00:13,70
2022-08-05 00:00:28,70
2022-08-05 00:00:33,69
...,...
2022-08-05 23:57:55,73
2022-08-05 23:58:10,73
2022-08-05 23:58:20,74


## Heart Rate

In [28]:
oneDate = pd.datetime(year = 2022, month = 8, day = 5)
oneDayData = auth2_client.intraday_time_series('activities/heart', oneDate, detail_level='1sec')
heart_intraday = pd.DataFrame(oneDayData["activities-heart-intraday"]["dataset"])
heart_intraday.head()

,time,value
0,00:00:03,67
1,00:00:08,68
2,00:00:13,70
3,00:00:28,70
4,00:00:33,69


### Floors
Probably not the best metric, but might provide some interesting insight

In [43]:
oneDayData = auth2_client.intraday_time_series("activities/floors", oneDate, detail_level='1min')
floors_intraday = pd.DataFrame(oneDayData["activities-floors-intraday"]["dataset"])
floors_intraday["time"] = pd.to_datetime(floors_intraday["time"])
floors_intraday.set_index("time",inplace=True)
floors_intraday[datetime(2022,8,6,7,15):][:30]

,value
time,
2022-08-06 07:15:00,0
2022-08-06 07:16:00,0
2022-08-06 07:17:00,0
2022-08-06 07:18:00,1
2022-08-06 07:19:00,1
2022-08-06 07:20:00,0
2022-08-06 07:21:00,0
2022-08-06 07:22:00,1
2022-08-06 07:23:00,1


### Elevation

In [41]:
oneDayData = auth2_client.intraday_time_series("activities/elevation", oneDate, detail_level='1min')
floors_elevation = pd.DataFrame(oneDayData["activities-elevation-intraday"]["dataset"])
floors_elevation["time"] = pd.to_datetime(floors_elevation["time"])
floors_elevation.set_index("time",inplace=True)
floors_elevation[datetime(2022,8,6,7,15):][:30]

,value
time,
2022-08-06 07:15:00,0
2022-08-06 07:16:00,0
2022-08-06 07:17:00,0
2022-08-06 07:18:00,10
2022-08-06 07:19:00,10
2022-08-06 07:20:00,0
2022-08-06 07:21:00,0
2022-08-06 07:22:00,10
2022-08-06 07:23:00,10


Looks like the floor data are just the elevation values divided by 10...